# Mineração - Aula 02

## Relatório sobre análise de reclamações

### Enunciado:
Utilizando o framework de análise descrito na aula passada e as técnicas de visualização apropriadas, faça uma análise sobre o dataset de reclamações do procon (link em anexo).
Eu quero saber quais empresas são mais odiadas.

Você deve entregar UM notebook jupyter com o relatório da sua análise. É para fazer descrições e visualizações, nada de fazer um monte de gráficos de qualquer jeito. Também não seja um robô, faça um relatório descrevendo seus pontos, os motivos que levam você a analisar ou concluir algo. Pense em setores, tipos de produtos, ... ou qualquer coisa que melhore meu entendimento sobre o problema. 

Assuma que sou uma besta acéfala que mal sabe ler (você não vai estar tão errado). Faça descrições cuidadosas e em "linguage humana".

### Dados:
https://www.kaggle.com/gerosa/procon/data

http://dados.gov.br/dataset/cadastro-nacional-de-reclamacoes-fundamentadas-procons-sindec1

#### Dicionario de dados:
http://dados.mj.gov.br/dataset/8ff7032a-d6db-452b-89f1-d860eb6965ff/resource/d87543d6-cf9d-4752-8f3c-1b0aa075dc45/download/dicionariodadossindec3-0.pdf

- Ano calendário de publicação do cadastro de reclamações fundamentadas:
2009: são 104.867 reclamações fundamentadas e arquivadas entre 09/2008 e 08/2009 de 39 Procons estaduais/municipais
2010: são 122.662 reclamações fundamentadas e arquivadas entre 09/2009 e 08/2010 de 52 Procons estaduais/municipais
2011: são 153.094 reclamações fundamentadas e arquivadas entre 01/2011 e 12/2011 de 67 Procons estaduais/municipais
2012: são 211.076 reclamações fundamentadas e arquivadas entre 01/2012 e 12/2012 de 112 Procons estaduais/municipais
- RadicalCNPJ: Aplica-se para pessoa jurídica e serve para agrupar as informações de um mesmo fornecedor (matriz e filiais), sendo os oitos primeiros dígitos do número do CNPJ - Exemplo: a matriz (central) do banco e suas filiais (agências)
- Faixa etária do consumidor distribuída da seguinte forma: até 20 anos, entre 21 e 30 anos, entre 31 e 40 anos, entre 41 e 50 anos, entre 51 e 60 anos, entre 61 e 70 anos, mais de 70 anos, Nao Informada (data de nascimento não informada no cadastro do consumidor), Não se aplica (são as reclamações (de ofício) em que o Procon é o reclamante)



### Passo 01 - Abrindo os arquivos com os dados a serem utilizados

In [1]:
%%time

## Mudar o endereço da pasta abaixo, caso queira mudar a pasta que receberá o arquivo pickle.
%cd C://_kopp//Dropbox//Filipe//_doutorado//Mineracao
from urllib.request import urlopen
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

pd.options.display.float_format = '{:.1f}'.format

C:\_kopp\Dropbox\Filipe\_doutorado\Mineracao
Wall time: 1.7 s


In [2]:
## Dados de população de cada UF serão usados para normalizar algumas análises.
## source: https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_popula%C3%A7%C3%A3o
populacao = {'AC': 831473, 'AL': 3378287, 'AP': 800567, 'AM': 4072343, 'BA': 15355449, 'CE': 9033796, 'DF': 3046233,
             'ES': 4019390, 'GO': 6792817, 'MA': 7004330, 'MT': 3350247, 'MS': 2718195, 'MG': 21140120, 'PA': 8377844,
             'PB': 4028416, 'PR': 11335726, 'PE': 9483440, 'PI': 3220991, 'RJ': 16736365, 'RN': 3512714, 'RS': 11333417,
             'RO': 1808986, 'RR': 524681, 'SC': 6994566, 'SP': 45162700, 'SE': 2292102, 'TO': 1553026}
print('população do Brasil:',sum(populacao.values()))

população do Brasil: 207908221


Os arquivos de dados do PROCON de 2009 a 2016 foram copiados para uma pasta Dropbox para poderem ser baixados automaticamente durante a execução do código. Na primeira vez que se executa o código, todos os arquivos são baixados, consolidados e salvos em um arquivo pickle. O arquivo pickle será utilizado nas futuras execuções com o objetivo de economizar tempo pois evita baixar os arquivos do PROCON novamente.

In [3]:
%%time
## Baixando os arquivos
urls = ['https://www.dropbox.com/s/lxq41kw10jwum6k/reclamacoes-fundamentadas-sindec-2009.csv?dl=1',
        'https://www.dropbox.com/s/48u53comem6vfqv/reclamacoes-fundamentadas-sindec-2010.csv?dl=1',
        'https://www.dropbox.com/s/ddsq2d9luyrelvd/reclamacoes-fundamentadas-sindec-2011.csv?dl=1',
        'https://www.dropbox.com/s/e9meajkvg3rb34v/reclamacoes-fundamentadas-sindec-2012.csv?dl=1',
        'https://www.dropbox.com/s/r89wzb1hmqyrzk7/reclamacoes-fundamentadas-sindec-2013.csv?dl=1',
        'https://www.dropbox.com/s/dcwtb62shu6zpws/reclamacoes-fundamentadas-sindec-2014.csv?dl=1',
        'https://www.dropbox.com/s/hsocwtkp53hon67/reclamacoes-fundamentadas-sindec-2015.csv?dl=1',
        'https://www.dropbox.com/s/4jx1g3aoze4lxmm/reclamacoes-fundamentadas-sindec-2016.csv?dl=1'] 
if os.path.isfile('reclama.pickle'):
    ## Se o arquivo pickle já existe, abrí-lo.
    print('pegando dados salvos no pickle')
    pd_reclama = pd.read_pickle('reclama.pickle')
else:
    ## Senão, baixar tudo e salvar em arquivo pickle.
    deli = 0
    pd_reclama = pd.DataFrame([])
    for url in urls:
        reclama_csv = urlopen(url)
        print('baixando dados de: ',url, end=' -> ')
        if deli < 3 : delim=';'
        else: delim = ','
        reclama_data = pd.read_csv(reclama_csv,delimiter=delim, dtype={0: str, 'NumeroCNPJ': str, 'RadicalCNPJ': str,
                                                                       'numeroCNPJ': str, 'radicalCNPJ': str,
                                                                       'CEPConsumidor': str}) 
        if deli < 3 : 
            reclama_data.rename(columns={'codigoregiao': 'CodigoRegiao','anocalendario': 'AnoCalendario',
                                         'radicalCNPJ': 'RadicalCNPJ', 'regiao': 'Regiao'}, inplace=True)
        deli += 1
        print(reclama_data.shape)
        pd_reclama = pd.concat([pd_reclama,reclama_data])
    pd_reclama.to_pickle('reclama.pickle')
print('total de linhas obtidas: ',pd_reclama.shape[0])
print('Arquivo pickle com ',int(os.stat('reclama.pickle').st_size/1024/1024)," MB.")

pegando dados salvos no pickle
total de linhas obtidas:  1586701
Arquivo pickle com  301  MB.
Wall time: 8.89 s


In [4]:
pd_reclama.dtypes

AnoCalendario             object
DataArquivamento          object
DataAbertura              object
CodigoRegiao             float64
Regiao                    object
UF                        object
strRazaoSocial            object
strNomeFantasia           object
Tipo                     float64
NumeroCNPJ               float64
RadicalCNPJ              float64
RazaoSocialRFB            object
NomeFantasiaRFB           object
CNAEPrincipal            float64
DescCNAEPrincipal         object
Atendida                  object
CodigoAssunto            float64
DescricaoAssunto          object
CodigoProblema           float64
DescricaoProblema         object
SexoConsumidor            object
FaixaEtariaConsumidor     object
CEPConsumidor             object
dtype: object

### Passo 02 - Organizando os dados

- Combinar colunas Razão Social com informações similares
- Normalizar a coluna Razão Social
- Corrigir SA e LTDA
- Diminuir tamaho das células em Faixa Etária

In [5]:
%%time
## Normalizando as Razões Sociais das empresas
pd_reclama['strRazaoSocial'].fillna(value=pd_reclama['RazaoSocialRFB'])
pd_reclama['strRazaoSocial'].fillna(value=pd_reclama['NomeFantasiaRFB'])
pd_reclama['strRazaoSocial'] = pd_reclama['strRazaoSocial'].str.replace('[^\w\s]','')
pd_reclama['strRazaoSocial'] = pd_reclama['strRazaoSocial'].str.strip()
pd_reclama['strRazaoSocial'] = pd_reclama['strRazaoSocial'].str.upper()
pd_reclama['strRazaoSocial'] = pd_reclama['strRazaoSocial'].str.normalize('NFKD').str.encode('ascii', 
                                                                errors='ignore').str.decode('utf-8')

## Padronizando SA e LTDA nos nomes das empresas
for s in [' S/A',' S.A',' SA.']:
    pd_reclama['strRazaoSocial'] = pd_reclama['strRazaoSocial'].str.replace(s,' SA')
for s in ['LTDA - ME','LTDA EPP','LTDA-EPP','LTDA- EPP','LTDA.','LTDAME']:
    pd_reclama['strRazaoSocial'] = pd_reclama['strRazaoSocial'].str.replace(s,'LTDA')

pd_reclama['FaixaEtariaConsumidor'] = pd_reclama['FaixaEtariaConsumidor'].str.replace('entre ','')
pd_reclama['FaixaEtariaConsumidor'] = pd_reclama['FaixaEtariaConsumidor'].str.replace(' anos','')

Wall time: 1min 27s


- Quebrar Data de Abertura e Data de Arquivamenteo em Ano e mês
- Calcular diferença entre datas em dias
- Criando variáveis binárias para sexo e atendimento
- Remover linhas sem CNPJ ou Razão Social. Remover reclamações que não são empresas (CPF).
- Remover colunas que não são mais usadas

In [6]:
%%time
## Ano da data de Arquivamento não é tão interessante quanto data de abertura da reclamação. Vamos trabalhar com ano e mês
## das duas datas e calcular a diferença entre elas em dias.
pd_reclama['DataAbertura'] = pd.to_datetime(pd_reclama['DataAbertura'],infer_datetime_format=True)
pd_reclama['ano_abertura'] = pd_reclama['DataAbertura'].dt.year
pd_reclama['mes_abertura'] = pd_reclama['DataAbertura'].dt.month

pd_reclama['DataArquivamento'] = pd.to_datetime(pd_reclama['DataArquivamento'],infer_datetime_format=True)
pd_reclama['ano_arquiva'] = pd_reclama['DataArquivamento'].dt.year
pd_reclama['mes_arquiva'] = pd_reclama['DataArquivamento'].dt.month

pd_reclama['diff'] = (pd_reclama['DataArquivamento'] - pd_reclama['DataAbertura']).dt.days


## Codificando Sexo e Atendipmento para 0 ou 1, para posteriormente calcular a média
pd_reclama['sexo_bin'] = np.nan
pd_reclama.loc[(pd_reclama['SexoConsumidor'] == 'M'),'sexo_bin'] = 1
pd_reclama.loc[(pd_reclama['SexoConsumidor'] == 'F'),'sexo_bin'] = 0
pd_reclama['atendimento_bin'] = (pd_reclama['Atendida'] == 'S')

## Mantendo somente as empresas e removendo colunas que não são mais necessárias
df_reclama = pd_reclama.copy()
#df_reclama.drop(df_reclama[df_reclama.Tipo < 1].index, inplace=True) 
df_reclama.dropna(subset=['NumeroCNPJ','strRazaoSocial'], inplace=True)
df_reclama.drop(columns=['NomeFantasiaRFB', 'RazaoSocialRFB','CodigoRegiao','CodigoAssunto','AnoCalendario','DataAbertura',
                         'DataArquivamento','CodigoProblema','Tipo','CEPConsumidor','CNAEPrincipal','strNomeFantasia'], 
                inplace=True)
print('total de linhas após limpeza: ',df_reclama.shape[0])
print('total de linhas removidas: ',pd_reclama.shape[0] - df_reclama.shape[0], ' ou ', 
      round((pd_reclama.shape[0] - df_reclama.shape[0])/pd_reclama.shape[0] *100,2) , '%')

total de linhas após limpeza:  1518886
total de linhas removidas:  67815  ou  4.27 %
Wall time: 1min


In [10]:
df_reclama.sort_values('NumeroCNPJ', inplace=True)
df_reclama

,Regiao,UF,strRazaoSocial,NumeroCNPJ,RadicalCNPJ,DescCNAEPrincipal,Atendida,DescricaoAssunto,DescricaoProblema,SexoConsumidor,FaixaEtariaConsumidor,ano_abertura,mes_abertura,ano_arquiva,mes_arquiva,diff,sexo_bin,atendimento_bin
97776,Sul,SC,BANCO DO BRASIL SA,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",S,Banco comercial,"SAC - Acesso ao serviço (onerosidade, problema...",F,21 a 30,2012.0,4.0,2012.0,6.0,70.0,0.0,True
80330,Sudeste,ES,BANCO DO BRASIL SA UNIDADE DE OUVIDORIA,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",N,Banco comercial,Não entrega de quitação/retenção de documentos,F,61 a 70,2011.0,1.0,2011.0,2.0,26.0,0.0,False
143107,Sudeste,MG,BANCO DO BRASIL SA,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",S,Cartão de Crédito,Cobrança indevida.,M,41 a 50,2010.0,11.0,2011.0,12.0,406.0,1.0,True
18845,Nordeste,MA,BANCO DO BRASIL SA,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",S,Banco comercial,Cobrança indevida.,M,21 a 30,2013.0,4.0,2013.0,7.0,93.0,1.0,True
230103,Nordeste,PE,BANCO DO BRASIL SA ENDUNICO PARA CIP ELETRONICA,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",N,"Transporte ( Rodoviário, Passageiros, Cargas, ...",Serviço não fornecido (entrega/instalação/não ...,F,41 a 50,2014.0,9.0,2014.0,12.0,83.0,0.0,False
14983,Nordeste,CE,BANCO DO BRASIL SA,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",S,Cartão de Crédito,Cobrança indevida.,F,51 a 60,2014.0,3.0,2014.0,7.0,126.0,0.0,True
48209,Nordeste,PB,BANCO DO BRASIL SA,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",N,Banco comercial,Cobrança indevida.,F,61 a 70,2011.0,8.0,2012.0,7.0,357.0,0.0,False
8206,Sul,SC,BANCO DO BRASIL SA,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",S,Cartão de Crédito,Cobrança indevida/abusiva,M,31 a 40,2016.0,8.0,2016.0,9.0,33.0,1.0,True
201756,Sudeste,SP,BANCO DO BRASIL,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",N,Banco comercial,Cobrança indevida/abusiva,M,até 20,2016.0,11.0,2016.0,12.0,35.0,1.0,False
37445,Nordeste,MA,BANCO DO BRASIL SA,191.0,0.0,"BANCOS MÚLTIPLOS, COM CARTEIRA COMERCIAL",S,Banco comercial,Transação Eletrônica Não Reconhecida,F,41 a 50,2014.0,9.0,2014.0,12.0,71.0,0.0,True


In [ ]:
%%time
## Deve-se criar um dataframe somente com o RadicalCNPJ e Razão Social, e remover as duplicatas do RadicalCNPJ, assim,
## obtém-se um nome para cada grupo empresarial (matriz e filial)

df_empresas = df_reclama[['RadicalCNPJ','strRazaoSocial']].copy()
df_empresas.dropna(subset=['RadicalCNPJ','strRazaoSocial'], inplace=True, how='any')
df_empresas.sort_values(['RadicalCNPJ','strRazaoSocial'], ascending=True, inplace=True)
df_empresas.drop_duplicates(subset=['RadicalCNPJ'], keep='first', inplace=True)
print('Foram obtidos',df_empresas.shape[0],'grupos empresariais distintos, através do Radical CNPJ')
print(df_empresas.head())

In [ ]:
%%time
df_resultado = pd.merge(df_reclama,df_empresas, on='RadicalCNPJ').sort_values(['RadicalCNPJ'])
df_resultado.rename(columns={'strRazaoSocial_y': 'razao','FaixaEtariaConsumidor': 'idade','SexoConsumidor': 'sexo'}, inplace=True)
#df_resultado.set_index(['razao'], inplace=True)
df_resultado.drop(columns=['strRazaoSocial_x', 'NumeroCNPJ','RadicalCNPJ'], inplace=True)

In [ ]:
df_resultado

## Passo 03 - Colocando os contadores

In [ ]:
df_resultado['razao'].groupby(df_resultado['Regiao']).describe()

In [ ]:
df_resultado['razao'].groupby(df_resultado['sexo_bin']).describe()

In [ ]:
df_resultado['razao'].groupby(df_resultado['sexo']).describe()

In [ ]:
df_resultado['razao'].groupby(df_resultado['Atendida']).describe()

In [ ]:
piores_empresas = df_reclama.groupby([ 'RadicalCNPJ','year'])['strRazaoSocial'].count().unstack()
piores_empresas = piores_empresas.sort_values(by=[2016,2015], ascending=False).head(25)
piores_empresas.drop(columns=range(2005,2013), inplace=True)
piores_empresas

In [ ]:
## Existe alguma diferença de reclamações por causa do sexo? 
## Aparentemente, o sexo Feminino 
agrupado_M = df_reclama[(df_reclama['SexoConsumidor'] == 'M') & (df_reclama['year'] >= 2012)].groupby([ 'month','year'])['strRazaoSocial'].count().unstack()
agrupado_F = df_reclama[(df_reclama['SexoConsumidor'] == 'F') & (df_reclama['year'] >= 2012)].groupby([ 'month','year'])['strRazaoSocial'].count().unstack()
sexo = agrupado_F/(agrupado_M + agrupado_F) * 100
print('Sexo feminino apresenta ',sexo.mean().mean(),'% das reclamações')
sexo.plot(figsize=size)

In [ ]:
agrupado_S = df_reclama[df_reclama['Atendida'] == 'S'].groupby([ 'month','year'])['strRazaoSocial'].count().unstack()
agrupado_N = df_reclama[df_reclama['Atendida'] == 'N'].groupby([ 'month','year'])['strRazaoSocial'].count().unstack()
atendida = agrupado_S/(agrupado_S + agrupado_N) * 100
atendida

## Passo 04 - Desenhando os gráficos

In [ ]:
labels = range(1,13)
size = (15,10)
radar.plot(figsize=size,)
plt.xticks(labels)
plt.xlabel("Meses")
plt.ylabel("Número de Reclamações")
plt.title('Frequência mensal de reclamações por ano')
plt.yticks([])
plt.show()

In [ ]:
piores_empresas.head(10).plot.barh(figsize=size, stacked=True)

In [ ]:
values = df_reclama[df_reclama['year'] == 2016]['RadicalCNPJ'].value_counts()
values